# SVASv


In [11]:
import asyncio
from typing import Optional

import sirius_sdk
import ipywidgets as widgets


################### P2P connection to Confidential-Storage Provider ###################
PROVIDER: Optional[sirius_sdk.Pairwise] = None
#######################################################


# Init connection to Wallet that maintained in the Cloud
# Off cource developer have alternative - he may maintain Wallet and secrets on other way (KERI library) 
# thanks to Sirius-SDK is friendly to Legacy and flexible due to dependency injection architecture 

cfg = sirius_sdk.Config().setup_cloud(
    server_uri='https://agents.socialsirius.com',
    credentials='s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rUj54zpQ9fWOm3lsdQwI3d2EzIARf+kcCNKU5wKJX9LY=',
    p2p={
        "their_verkey": "FBHVE36ZfixjBwqJWKy45sTRJdt9KDFrqQPL6rExJweN",
        "my_keys": [
          "22UJ1f3nPuJR9y6znPXwGw2xEyzmNRigwbfNoJPraPtY",
          "5JjNff6y8GxWtWxWA5BiKkfZVx4hYQHscwZt7z613CrENWL7xRMehy5MWNy6BqooX73PmDDSso9npNEqsmbXgXCG"
        ]
   }
)

async def establish_connection():
    try:
        async with sirius_sdk.context(cfg):
            conn_key = await sirius_sdk.Crypto.create_key()
            endpoint = [e for e in await sirius_sdk.endpoints() if not e.routing_keys][0]
            invitation = sirius_sdk.aries_rfc.Invitation(
                label='Vault Client',
                endpoint=endpoint.address, 
                recipient_keys=[conn_key],
                routing_keys=endpoint.routing_keys
            )
            invitation_url = 'https://example.com/invitation' + invitation.invitation_url
            output.append_stdout('\n !!!!!! Copy invitation and PASS to Provider script !!!!!!\n')
            output.append_stdout('_'*128 + '\n')
            output.append_stdout(invitation_url)
            output.append_stdout('\n' + '_'*128)
            # Start ti listen connection requests
            output.append_stdout('\nWait for connection requests ...')
            listener = await sirius_sdk.subscribe()
            async for event in listener:
                msg = event.message
                if isinstance(msg, sirius_sdk.aries_rfc.ConnRequest) and event.recipient_verkey == conn_key:
                    output.append_stdout('\nReceived conn-request, start rfc-160 to establish connection...')
                    did, verkey = await sirius_sdk.DID.create_and_store_my_did()
                    output.append_stdout(f'\nMy DID: {did}  My Verkey: {verkey}  My Endpoint: {endpoint.address}')
                    ########
                    rfc_0160 = sirius_sdk.aries_rfc.Inviter(
                        me=sirius_sdk.Pairwise.Me(did, verkey),
                        connection_key=conn_key,
                        my_endpoint=endpoint
                    )
                    success, p2p = await rfc_0160.create_connection(request=msg)
                    if success:
                        their_label = p2p.their.label
                        output.append_stdout('_'*128 + '\n')
                        output.append_stdout(f'\n!!! Connection with "{their_label}" was established !!!!')
                        output.append_stdout('\n' + '_'*128)
                    else:
                        output.append_stderr(rfc_0160.problem_report.explain)
                    ########
    except Exception as e:
        output.append_stderr(repr(e))

output = widgets.Output()
button = widgets.Button(description="Generate Invitation", button_style='primary')

def on_button_clicked(b):
    output.clear_output()
    PROVIDER = None
    asyncio.ensure_future(establish_connection())
button.on_click(on_button_clicked)
        
display(button, output)


Button(button_style='primary', description='Generate Invitation', style=ButtonStyle())

Output()